### Initialization

In [1]:
import glob
import json
from nltk import tokenize
# import textdistance
import random
import string
import os
import numpy
import pandas
import fasttext
import gensim
# import mxnet as mx
# from mxnet import gluon
# import gluonnlp as nlp
# from elmoformanylangs import Embedder

In [6]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
def randomTokens():
    """Generate a random string of fixed length """
    tokensLength = int(random.gauss(4,2))+1
    letters = string.ascii_lowercase
    tokens = []
    for _ in range(tokensLength):
        stringLength = int(random.gauss(6,3)+1)
        tokens.append(''.join(random.choice(letters) for i in range(stringLength)))
    return tokens

### Subject extraction

In [3]:
class SubjectExtractor:
    
    def extract(self):
        return ['ahoj jak se mas']

class RandomSubjectExtractor(SubjectExtractor):
    
    def extract(self):
        subjects_num = int(numpy.random.exponential())+1
        return [randomTokens() for _ in range(subjects_num)]
    
class ReferenceSubjectExtractor(SubjectExtractor):
    
    _REF_FILENAME = '_ref.json'
    _path = None
    _ref = None
    _ref_subjects = None
    
    def __init__(self, path):
        self._path = path
    
    def extract(self):
        filename = os.path.join(self._path, self._REF_FILENAME)
        with open(filename, encoding='utf-8') as f:
            data = f.read()
            self._ref = json.loads(data)
            self._ref_subjects = [item['name'] for item in self._ref['subject']['items']]
        return self._ref_subjects

### EvaluationMachine

In [4]:
class EvaluationMachine:
    
    _core = None
    _subjects = None
    _ref_subjects = None
    _similarity_matrix = None
    
    def __init__(self, core=None):
        self._core = core
    
    def evaluateSubjects(self, subjects, ref_subjects):
        self._subjects = [' '.join(s) if isinstance(s, list) else s for s in subjects]
        self._ref_subjects = [' '.join(s) if isinstance(s, list) else s for s in ref_subjects]
        return self.evaluate()
    
    def evaluate(self):
        self.compute_matrix()
        return self._similarity_matrix
        
    def compute_matrix(self):
        sl = len(self._subjects)
        rl = len(self._ref_subjects)
        result_matrix = numpy.zeros((sl, rl))
        for i in range(sl):
            for j in range(rl):
                d = self._core.compute(self._subjects[i], self._ref_subjects[j])
                result_matrix[i][j] = d
        self._similarity_matrix = result_matrix
        
    def display(self, with_headers=False):
        if with_headers:
            df = pandas.DataFrame(self._similarity_matrix, columns=self._ref_subjects, index=self._subjects)
        else:
            df = pandas.DataFrame(self._similarity_matrix)
        display(df)
        
    
class RandomEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(RandomSimilarityMachine())
        
class JaccardEvaluationMachine(EvaluationMachine):

    def __init__(self, core=None):
        super().__init__(JaccardSimilarityMachine())
        
class FastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(FastTextSimilarityMachine(core))
        
class GensimFastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(GensimFastTextSimilarityMachine(core))

### Similarity functions

In [5]:
class SimilarityMachine:
    
    _text = None
    _reference = None
    _model = None
    
    def __init__(self, model=None):
        self._model = model
        
    def init_case(self, text, reference):
        self._text = text
        self._reference = reference
        
    def preprocess(self):
        None
    
    def compute(self, text, reference):
        self.init_case(text, reference)
        self.preprocess()
        return self._inner_compute()
    
    def _inner_compute(self):
        return 0.5
    
class RandomSimilarityMachine(SimilarityMachine):
    
    def _inner_compute(self):
        return random.random()
    
class JaccardSimilarityMachine(SimilarityMachine):
    
    def preprocess(self):
        if isinstance(self._text, str):
            self._text = self._text.split()
        if isinstance(self._reference, str):
            self._reference = self._reference.split()        

    def _inner_compute(self):
        return textdistance.jaccard(self._text, self._reference)

class SorensenDiceSimilarityMachine(JaccardSimilarityMachine):
    
    def _inner_compute(self):
        return textdistance.sorensen_dice(self._text, self._reference)

class FastTextSimilarityMachine(SimilarityMachine):
    
    def __init__(self, model=None):
        if model is None:
            path = 'fasttext/cc.cs.300.bin'
            print('Loading fasttext model from ' + path)
            model = fasttext.load_model(path)
        super().__init__(model)
    
    def _inner_compute(self):
        return 1/(numpy.linalg.norm(self._model[self._text] - self._model[self._reference])+1)
    
class GensimFastTextSimilarityMachine(SimilarityMachine):
    
    def __init__(self, model=None):
        if model is None:
            path = 'fasttext/cc.cs.300.bin'
            print('Loading fasttext model from ' + path)
            model = gensim.models.FastText.load_fasttext_format(path)
        super().__init__(model)
    
    def _inner_compute(self):
        return 1/(self._model.wmdistance(self._text, self._reference)+1)

In [7]:
vzdirs = glob.glob('../dokumenty/*/*/*')

### Measures

### Playground

In [6]:
vzdir = '../test-data/test_src/test_sub_01/test_contr_01'

In [7]:
subjects = [
    'stavební práce',
    'montáž plošiny'
]

In [14]:
refExtr = ReferenceSubjectExtractor(vzdir)
refSubjects = refExtr.extract()
refSubjects

['zpracování projektové dokumentace a vydání staveního povolení',
 'stavební práce',
 'dodávka a montáž plošiny']

In [15]:
extr = RandomSubjectExtractor()
subjects = extr.extract()
subjects

[['nbtwzdanuq', 'spioogvb', 'uwoky', 'odp', 'azcicxmf']]

In [16]:
evalMachine = RandomEvaluationMachine()

In [17]:
evalMachine.evaluateSubjects(subjects, refSubjects)

array([[0.00403215, 0.90746506, 0.43651078]])

In [18]:
evalMachine.display(True)

,zpracování projektové dokumentace a vydání staveního povolení,stavební práce,dodávka a montáž plošiny
nbtwzdanuq spioogvb uwoky odp azcicxmf,0.004032,0.907465,0.436511


In [18]:
evalMachine = JaccardEvaluationMachine()

In [19]:
evalMachine.evaluateSubjects(subjects, refSubjects)

array([[0. , 1. , 0. ],
       [0. , 0. , 0.5]])

In [20]:
evalMachine.display(True)

,zpracování projektové dokumentace a vydání staveního povolení,stavební práce,dodávka a montáž plošiny
stavební práce,0.0,1.0,0.0
montáž plošiny,0.0,0.0,0.5


In [19]:
fasttextEval = FastTextEvaluationMachine()
fasttextEval

Loading fasttext model from fasttext/cc.cs.300.bin


In [20]:
fasttextEval.evaluateSubjects(subjects, refSubjects)

array([[0.88092765, 0.79895507, 0.81048434]])

In [21]:
fasttextEval.display(True)

,zpracování projektové dokumentace a vydání staveního povolení,stavební práce,dodávka a montáž plošiny
nbtwzdanuq spioogvb uwoky odp azcicxmf,0.880928,0.798955,0.810484


In [25]:
gensimFTEval = GensimFastTextEvaluationMachine()
gensimFTEval

2019-11-11 00:11:00,578 INFO: loading 2000000 words for fastText model from fasttext/cc.cs.300.bin


Loading fasttext model from fasttext/cc.cs.300.bin


2019-11-11 00:23:49,056 INFO: loading weights for 2000000 words for fastText model from fasttext/cc.cs.300.bin
2019-11-11 00:27:13,913 INFO: loaded (2000000, 300) weight matrix for fastText model from fasttext/cc.cs.300.bin


In [26]:
gensimFTEval.evaluateSubjects(subjects, refSubjects)

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
2019-11-11 00:27:14,991 INFO: Removed 1 and 6 OOV words from document 1 and 2 (respectively).
2019-11-11 00:27:14,998 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-11 00:27:15,004 INFO: built Dictionary(23 unique tokens: ['a', 'b', 'c', 'e', 'n']...) from 2 documents (total 68 corpus positions)
2019-11-11 00:27:15,075 INFO: Removed 1 and 1 OOV words from document 1 and 2 (respectively).
2019-11-11 00:27:15,077 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-11 00:27:15,078 INFO: built Dictionary(12 unique tokens: ['a', 'b', 'c', 'e', 'n']...) from 2 documents (total 26 corpus positions)
2019-11-11 00:27:15,089 INFO: Removed 1 and 3 OOV words from document 1 and 2 (respectively).
2019-11-11 00:27:15,089 INFO: adding document #

array([[0.47409022, 1.        , 0.36634691],
       [0.38345893, 0.34563655, 0.53334089]])

In [27]:
gensimFTEval.display(True)

,zpracování projektové dokumentace a vydání staveního povolení,stavební práce,dodávka a montáž plošiny
stavební práce,0.474090,1.000000,0.366347
montáž plošiny,0.383459,0.345637,0.533341


In [30]:
ftmodel = gensimFTEval._core._model

In [41]:
ftmodel.wmdistance('Miloš Zeman', 'Praha')

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-13 15:34:38,074 INFO: Removed 1 and 0 OOV words from document 1 and 2 (respectively).
2019-11-13 15:34:38,077 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-13 15:34:38,080 INFO: built Dictionary(13 unique tokens: ['M', 'Z', 'a', 'e', 'i']...) from 2 documents (total 15 corpus positions)


2.4152138310925486

In [8]:
ftmodel2 = fasttext.load_model('fasttext/cc.cs.300.bin')
ftmodel2['paper']

array([-0.05804337, -0.03683203, -0.01250542,  0.01675747,  0.06488278,
        0.0722379 ,  0.02369922, -0.03951797,  0.0249566 , -0.09510952,
        0.05870747,  0.03974752,  0.00672182, -0.00674034, -0.06719575,
        0.0631699 , -0.02813651, -0.03256127, -0.0369756 , -0.00069492,
        0.08782576,  0.00062506, -0.00337796, -0.05085486, -0.06829706,
        0.04374601,  0.01159262, -0.08052534, -0.05211427, -0.04039497,
       -0.08473791, -0.03656073,  0.01781434, -0.10210821,  0.08074468,
       -0.02363156,  0.01540139,  0.05923993, -0.04091276,  0.09380479,
       -0.0570371 , -0.01719799, -0.09991248,  0.20365804, -0.13507062,
       -0.04401433, -0.08736669,  0.10895465, -0.0104823 ,  0.03327014,
       -0.00188187, -0.0121702 ,  0.11516406,  0.13058259,  0.06168722,
        0.0542936 ,  0.07715475,  0.06375038,  0.09789898,  0.09845754,
       -0.01105744,  0.00291643, -0.00809658, -0.04786212, -0.06753337,
        0.03914187, -0.138804  , -0.12774865,  0.00028894, -0.07

In [24]:
ftmodel2.get_word_vector('pes')

array([ 3.50985974e-02, -5.04605472e-03, -3.19778845e-02, -1.27138332e-01,
        6.54838160e-02, -2.35059354e-02,  2.33508229e-01, -2.17888355e-02,
       -2.19157338e-01,  1.16614953e-01, -8.44471455e-02, -8.51622447e-02,
       -4.71264981e-02,  1.27684444e-01, -9.18781906e-02, -1.50643975e-01,
        1.22504547e-01,  8.26932788e-02,  8.27983394e-02, -8.30178112e-02,
       -1.21429674e-02,  5.07118218e-02,  6.97170198e-03, -1.69049148e-02,
       -6.02188446e-02, -1.14216186e-01, -2.02800602e-01,  1.75772738e-02,
       -3.10905203e-02, -2.98842639e-02, -2.57130146e-01,  8.87810886e-02,
        1.25467218e-02, -1.39928788e-01,  1.48580834e-01,  3.87761667e-02,
       -4.27381434e-02,  5.04903495e-03,  1.24494880e-01,  4.74659577e-02,
       -1.14616990e-01, -1.00901842e-01,  2.84647904e-02,  1.85851216e-01,
        1.17499620e-01,  8.86817053e-02, -2.28305936e-01, -8.30656365e-02,
       -1.01898253e-01, -7.16145039e-02,  2.15689898e-01, -1.80715859e-01,
       -4.38175946e-02,  

In [23]:
ftmodel2.get_sentence_vector('pes')

array([ 1.70342829e-02, -2.44898442e-03, -1.55197177e-02, -6.17036112e-02,
        3.17810364e-02, -1.14080552e-02,  1.13327749e-01, -1.05747012e-02,
       -1.06362887e-01,  5.65963387e-02, -4.09844443e-02, -4.13315035e-02,
       -2.28717420e-02,  6.19686544e-02, -4.45909277e-02, -7.31115267e-02,
        5.94547130e-02,  4.01332453e-02,  4.01842371e-02, -4.02907506e-02,
       -5.89330494e-03,  2.46117953e-02,  3.38355242e-03, -8.20440426e-03,
       -2.92258058e-02, -5.54321520e-02, -9.84245241e-02,  8.53071827e-03,
       -1.50890565e-02, -1.45036280e-02, -1.24792099e-01,  4.30878215e-02,
        6.08925754e-03, -6.79111630e-02,  7.21102282e-02,  1.88191049e-02,
       -2.07419582e-02,  2.45043100e-03,  6.04206771e-02,  2.30364911e-02,
       -5.56266718e-02, -4.89703454e-02,  1.38147194e-02,  9.01985392e-02,
        5.70256896e-02,  4.30395901e-02, -1.10802941e-01, -4.03139628e-02,
       -4.94539328e-02, -3.47564220e-02,  1.04680039e-01, -8.77062082e-02,
       -2.12658439e-02,  

In [25]:
ftmodel2.get_sentence_vector('mám hezkého žlutého psa')

array([ 4.00275216e-02,  2.81146914e-02,  1.33901704e-02, -1.82476081e-02,
       -4.65330109e-02, -5.13640465e-03,  1.01567067e-01, -2.44819950e-02,
        2.54845116e-02, -1.04111852e-02,  6.91427365e-02,  1.81085197e-03,
       -1.83291268e-02,  2.95593496e-02, -3.10038142e-02, -7.11458623e-02,
        1.72886774e-02,  2.77961828e-02, -4.17141691e-02, -6.32420108e-02,
        3.18842903e-02,  1.04474593e-02, -2.79363357e-02,  9.81238950e-03,
        5.80555163e-02, -7.59777799e-03, -4.22276333e-02,  5.36541268e-02,
       -3.77881937e-02, -1.18482873e-01, -3.04954406e-02,  4.59765680e-02,
       -2.99367569e-02, -1.03782024e-02,  3.90904304e-03, -3.39209065e-02,
        2.22609192e-02,  7.32923253e-03, -9.89848655e-03,  9.23861191e-03,
       -3.81879918e-02, -5.42033762e-02, -2.92862114e-02,  5.32729328e-02,
        3.94654684e-02,  6.49303105e-03, -1.35319345e-02, -3.34074837e-03,
       -3.73623185e-02, -2.41040299e-03,  3.76058370e-02, -8.91952403e-03,
       -1.02717746e-02,  

In [26]:
ftmodel2.get_word_vector('mám hezkého žlutého psa')

array([ 6.11417636e-05,  8.23663827e-03, -5.22053801e-04, -5.11732604e-03,
       -6.82476023e-03, -4.44123428e-03,  1.21401297e-02, -4.04047692e-04,
        2.38102698e-03, -5.20278700e-04, -1.03039318e-03,  3.11700394e-04,
        1.17594087e-02, -9.82082356e-03, -8.12636968e-03,  4.03888756e-03,
       -5.75271202e-03,  4.57962835e-03, -1.67881616e-03, -1.63012408e-02,
        1.00360699e-02, -6.40457962e-03, -9.74510238e-03,  9.74437315e-03,
        1.26562379e-02,  2.20758119e-03,  1.21648933e-04, -6.47571404e-04,
        3.72477737e-03, -1.32272318e-02,  6.94467546e-03, -7.28271483e-03,
       -1.62063278e-02,  3.58626782e-03,  4.58025327e-03, -1.06557822e-02,
        7.57489179e-04,  1.35812405e-02, -3.81398387e-03,  8.99256766e-03,
       -9.28559247e-03, -3.71225551e-03, -1.90083757e-02,  1.33154579e-02,
       -5.33293514e-03,  6.43207924e-03, -3.99484113e-03,  9.19916853e-03,
        8.54840782e-03,  1.38443429e-02,  1.51529219e-02, -2.12390884e-03,
        7.10761966e-03,  

In [9]:
numpy.linalg.norm(ftmodel['automobil'] - ftmodel['osobní vůz'])

NameError: name 'ftmodel' is not defined

In [2]:
ftmodel = gensim.models.FastText.load_fasttext_format('fasttext/cc.cs.300.bin')
ftmodel

C:\x\diplomka\research\researchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [11]:
ftmodel['pes']

C:\x\diplomka\research\researchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 3.50985974e-02, -5.04605472e-03, -3.19778845e-02, -1.27138332e-01,
        6.54838160e-02, -2.35059354e-02,  2.33508229e-01, -2.17888355e-02,
       -2.19157338e-01,  1.16614953e-01, -8.44471455e-02, -8.51622447e-02,
       -4.71264981e-02,  1.27684444e-01, -9.18781906e-02, -1.50643975e-01,
        1.22504547e-01,  8.26932788e-02,  8.27983394e-02, -8.30178112e-02,
       -1.21429674e-02,  5.07118218e-02,  6.97170198e-03, -1.69049148e-02,
       -6.02188446e-02, -1.14216186e-01, -2.02800602e-01,  1.75772738e-02,
       -3.10905203e-02, -2.98842639e-02, -2.57130146e-01,  8.87810886e-02,
        1.25467218e-02, -1.39928788e-01,  1.48580834e-01,  3.87761667e-02,
       -4.27381434e-02,  5.04903495e-03,  1.24494880e-01,  4.74659577e-02,
       -1.14616990e-01, -1.00901842e-01,  2.84647904e-02,  1.85851216e-01,
        1.17499620e-01,  8.86817053e-02, -2.28305936e-01, -8.30656365e-02,
       -1.01898253e-01, -7.16145039e-02,  2.15689898e-01, -1.80715859e-01,
       -4.38175946e-02,  

In [6]:
ftmodel['mám hezkého žlutého psa']

C:\x\diplomka\research\researchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 6.11417636e-05,  8.23663827e-03, -5.22053801e-04, -5.11732604e-03,
       -6.82476023e-03, -4.44123382e-03,  1.21401297e-02, -4.04047692e-04,
        2.38102698e-03, -5.20278700e-04, -1.03039318e-03,  3.11700394e-04,
        1.17594087e-02, -9.82082356e-03, -8.12636968e-03,  4.03888756e-03,
       -5.75271202e-03,  4.57962835e-03, -1.67881604e-03, -1.63012408e-02,
        1.00360699e-02, -6.40457962e-03, -9.74510238e-03,  9.74437315e-03,
        1.26562370e-02,  2.20758119e-03,  1.21648933e-04, -6.47571404e-04,
        3.72477737e-03, -1.32272318e-02,  6.94467546e-03, -7.28271483e-03,
       -1.62063278e-02,  3.58626782e-03,  4.58025327e-03, -1.06557822e-02,
        7.57489179e-04,  1.35812405e-02, -3.81398387e-03,  8.99256766e-03,
       -9.28559154e-03, -3.71225527e-03, -1.90083757e-02,  1.33154579e-02,
       -5.33293514e-03,  6.43207924e-03, -3.99484113e-03,  9.19916760e-03,
        8.54840688e-03,  1.38443429e-02,  1.51529210e-02, -2.12390884e-03,
        7.10761966e-03,  

In [10]:
ftmodel.wv['pes']

array([ 3.50985974e-02, -5.04605472e-03, -3.19778845e-02, -1.27138332e-01,
        6.54838160e-02, -2.35059354e-02,  2.33508229e-01, -2.17888355e-02,
       -2.19157338e-01,  1.16614953e-01, -8.44471455e-02, -8.51622447e-02,
       -4.71264981e-02,  1.27684444e-01, -9.18781906e-02, -1.50643975e-01,
        1.22504547e-01,  8.26932788e-02,  8.27983394e-02, -8.30178112e-02,
       -1.21429674e-02,  5.07118218e-02,  6.97170198e-03, -1.69049148e-02,
       -6.02188446e-02, -1.14216186e-01, -2.02800602e-01,  1.75772738e-02,
       -3.10905203e-02, -2.98842639e-02, -2.57130146e-01,  8.87810886e-02,
        1.25467218e-02, -1.39928788e-01,  1.48580834e-01,  3.87761667e-02,
       -4.27381434e-02,  5.04903495e-03,  1.24494880e-01,  4.74659577e-02,
       -1.14616990e-01, -1.00901842e-01,  2.84647904e-02,  1.85851216e-01,
        1.17499620e-01,  8.86817053e-02, -2.28305936e-01, -8.30656365e-02,
       -1.01898253e-01, -7.16145039e-02,  2.15689898e-01, -1.80715859e-01,
       -4.38175946e-02,  

In [92]:
ftmodel.init_sims(True)

2019-11-09 13:26:57,996 INFO: precomputing L2-norms of word weight vectors
2019-11-09 13:27:59,903 INFO: precomputing L2-norms of ngram weight vectors


In [93]:
ftmodel.wmdistance('rozbitý automobil', 'porouchaný osobní vůz')

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 13:31:51,501 INFO: Removed 1 and 2 OOV words from document 1 and 2 (respectively).
2019-11-09 13:31:51,520 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-09 13:31:51,525 INFO: built Dictionary(19 unique tokens: ['a', 'b', 'i', 'l', 'm']...) from 2 documents (total 35 corpus positions)


0.43326079731659234

In [94]:
ftmodel.wmdistance(['rozbitý automobil'], ['porouchaný osobní vůz'])

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 13:31:53,789 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-09 13:31:53,790 INFO: built Dictionary(2 unique tokens: ['rozbitý automobil', 'porouchaný osobní vůz']) from 2 documents (total 2 corpus positions)


0.3405744433403015

In [40]:
prague = 'Praha (německy Prag; v jiných jazycích často Prague či Praga) je hlavní a současně největší město Česka a 15. největší město Evropské unie. Leží mírně na sever od středu Čech na řece Vltavě, uvnitř Středočeského kraje, jehož je správním centrem, ale jako samostatný kraj není jeho součástí. Je sídlem velké části státních institucí a množství dalších organizací a firem. Sídlí zde prezident republiky, parlament, vláda, ústřední státní orgány a jeden ze dvou vrchních soudů. Mimoto je Praha sídlem řady dalších úřadů, jak ústředních, tak i územních samosprávných celků; sídlí zde též ústředí většiny politických stran, hnutí a iniciativ a centrály téměř všech církví, náboženských a dalších sdružení s celorepublikovou působností registrovaných v ČR. '
berlin = 'Berlín (německy Berlin) je hlavní město Německa a zároveň i spolková země Spolkové republiky Německo. Hlavním městem Německa se stal roku 1991 a od sjednocení Německa (a tím i obou částí města) Berlín patří k největším městům v Evropě a je druhým největším městem Evropské unie. Žije zde přibližně 3,613 milionu[1] obyvatel. '
zeman = 'Miloš Zeman (* 28. září 1944 Kolín) je český politik, ekonom, prognostik a třetí prezident České republiky. Úřadu se poprvé ujal složením slibu 8. března 2013 a druhé funkční období zahájil přesně o pět let později. V období 1993–2001 působil jako předseda České strany sociálně demokratické. V letech 1996–1998 zastával funkci předsedy Poslanecké sněmovny Parlamentu České republiky a následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru, jejíž existence byla umožněna na základě tzv. „opoziční smlouvy“ s Občanskou demokratickou stranou. '

In [52]:
prague_cleared = 'praha německy prag v jiných jazycích často prague praga je hlavní současně největší město česka 15. největší město evropské unie leží mírně sever středu čech řece vltavě uvnitř středočeského kraje je správním centrem samostatný kraj není součástí je sídlem velké části státních institucí množství dalších organizací firem sídlí zde prezident republiky parlament vláda ústřední státní orgány jeden dvou vrchních soudů je praha sídlem řady dalších úřadů ústředních územních samosprávných celků sídlí zde ústředí většiny politických stran hnutí iniciativ centrály všech církví náboženských dalších sdružení celorepublikovou působností registrovaných v čr'
berlin_cleared = 'berlín německy berlin je hlavní město německa zároveň spolková země spolkové republiky německo hlavním městem německa se stal roku 1991 sjednocení německa obou částí města berlín patří největším městům v evropě je druhým největším městem evropské unie žije zde přibližně 3,613 milionu obyvatel'
zeman_cleared = 'miloš zeman * 28. září 1944 kolín je český politik ekonom prognostik třetí prezident české republiky úřadu se poprvé ujal složením slibu 8. března 2013 druhé funkční období zahájil přesně pět let později'
zeman_cleared_long = 'miloš zeman * 28. září 1944 kolín je český politik ekonom prognostik třetí prezident české republiky úřadu se poprvé ujal složením slibu 8. března 2013 druhé funkční období zahájil přesně pět let později období 1993-2001 působil předseda české strany sociálně demokratické letech 1996–1998 zastával funkci předsedy poslanecké sněmovny parlamentu české republiky následující čtyři roky byl předsedou sociálnědemokratické vlády menšinového charakteru existence byla umožněna základě opoziční smlouvy občanskou demokratickou stranou'

In [95]:
ftmodel.wmdistance(prague_cleared, berlin_cleared)

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 13:32:04,236 INFO: Removed 88 and 42 OOV words from document 1 and 2 (respectively).
2019-11-09 13:32:04,238 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-09 13:32:04,243 INFO: built Dictionary(42 unique tokens: ['.', '1', '5', 'a', 'b']...) from 2 documents (total 814 corpus positions)


0.19701468985040538

In [96]:
ftmodel.wmdistance(tokenize.word_tokenize(prague_cleared), tokenize.word_tokenize(berlin_cleared))

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 13:32:12,389 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-09 13:32:12,392 INFO: built Dictionary(105 unique tokens: ['15.', 'celků', 'celorepublikovou', 'centrem', 'centrály']...) from 2 documents (total 132 corpus positions)


0.9690416386421717

In [97]:
ftmodel.wmdistance(berlin_cleared, zeman_cleared_long)

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 13:32:22,136 INFO: Removed 42 and 68 OOV words from document 1 and 2 (respectively).
2019-11-09 13:32:22,137 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-09 13:32:22,140 INFO: built Dictionary(49 unique tokens: [',', '1', '3', '6', '9']...) from 2 documents (total 712 corpus positions)


0.19634454204361407

In [99]:
ftmodel.wmdistance(tokenize.word_tokenize(berlin_cleared), tokenize.word_tokenize(zeman_cleared_long))

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 13:32:48,876 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-09 13:32:48,878 INFO: built Dictionary(99 unique tokens: ['1991', '3,613', 'berlin', 'berlín', 'druhým']...) from 2 documents (total 112 corpus positions)


1.1316418169048859

In [100]:
ftmodel.wmdistance(tokenize.word_tokenize(prague_cleared), tokenize.word_tokenize(zeman_cleared_long))

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 13:33:00,498 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-09 13:33:00,500 INFO: built Dictionary(139 unique tokens: ['15.', 'celků', 'celorepublikovou', 'centrem', 'centrály']...) from 2 documents (total 158 corpus positions)


1.1357497102982925

In [101]:
ftmodel.wmdistance(['václav','havel'], ['prague'])

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 13:33:05,758 INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-11-09 13:33:05,759 INFO: built Dictionary(3 unique tokens: ['havel', 'václav', 'prague']) from 2 documents (total 3 corpus positions)


1.1472067816445828

In [58]:
ftmodel.most_similar(['zeman'])

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2019-11-09 10:59:06,354 INFO: precomputing L2-norms of word weight vectors
2019-11-09 11:02:26,799 INFO: precomputing L2-norms of ngram weight vectors
c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('zemané', 0.6708066463470459),
 ('zemanovský', 0.6586387157440186),
 ('zemana', 0.6548796892166138),
 ('zemanovec', 0.6425000429153442),
 ('zemanů', 0.6334505677223206),
 ('zemanský', 0.6306949853897095),
 ('zemanům', 0.6305753588676453),
 ('zemanovská', 0.6292722225189209),
 ('zemanovské', 0.6257205009460449),
 ('zemanovcům', 0.6248170137405396)]

In [59]:
ftmodel.most_similar(['Zeman'])

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('gtmZeman', 0.7986787557601929),
 ('.Zeman', 0.7924207448959351),
 ('Zeman.', 0.782761812210083),
 ('M.Zeman', 0.7652435302734375),
 ('58Zeman', 0.7376464009284973),
 ('25Zeman', 0.7279771566390991),
 ('hlasůZeman', 0.7132811546325684),
 ('Sobotka', 0.7005442380905151),
 ('Zemanovec', 0.6974204778671265),
 ('Zemana', 0.6958152651786804)]

In [60]:
ftmodel.most_similar(['Miloš Zeman'])

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('gtmZeman', 0.6926430463790894),
 ('DostálováMasérkaJindra', 0.6825854182243347),
 ('25Zeman', 0.6752279996871948),
 ('Karlštějn', 0.6739051342010498),
 ('Machovský-Vrbka', 0.6715472936630249),
 ('Karlštej', 0.6706434488296509),
 ('KarlštejnKLADNOMěstské', 0.664710283279419),
 ('boudyKněžiceKocbeřeKošovKošťálovKřenovKřížliceKundratice',
  0.6556413173675537),
 ('SobieskiVojtaVoky29VšelicosWoxyszoe1997ZorinkaZuz', 0.6530987024307251),
 ('Karlštejn.Další', 0.6529257893562317)]

In [61]:
ftmodel.most_similar(['Miloš', 'Zeman'])

c:\users\mvancl\documents\temp\diplomka\research\pcrenv64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Zeman.', 0.73337721824646),
 ('gtmZeman', 0.6753553152084351),
 ('.Zeman', 0.6681327819824219),
 ('M.Zeman', 0.6440125703811646),
 ('Klaus.', 0.6333649158477783),
 ('Havel', 0.6274571418762207),
 ('58Zeman', 0.6218777894973755),
 ('ZemanMiloš', 0.62071692943573),
 ('25Zeman', 0.610581636428833),
 ('Zemanec', 0.6085720658302307)]

In [105]:
e = Embedder(model_dir='C:\\Users\\mvancl\\Documents\\temp\\diplomka\\research\\elmo\\139')

2019-11-08 12:25:48,675 INFO: char embedding size: 2710
2019-11-08 12:25:50,104 INFO: word embedding size: 281663
2019-11-08 12:25:54,819 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(281663, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2710, 50, padding_idx=2707)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_fe

In [433]:
sents = [['po', 'silnici', 'jede', 'zelený', 'bagr'],
         ['po', 'cestě', 'jede', 'zelený', 'traktor'],
        ['zelený', 'bagr', 'jede', 'po', 'silnici'],
        ['nechci', 'se', 'na', 'to', 'dívat']]

In [431]:
sents = [['po silnici jede zelený bagr'],
         ['po cestě jede zelený traktor'],
        ['zelený bagr jede po silnici'],
        ['nechci se na to dívat']]

In [348]:
sents = [['silnice'], ['silnice']]

In [273]:
elmovecs = e.sents2elmo(sents)

2019-11-08 13:43:16,936 INFO: 1 batches, avg len: 3.0


In [434]:
elmovecs = e.sents2elmo(sents)
numpy.linalg.norm(elmovecs[0] - elmovecs[1])

2019-11-08 13:56:02,087 INFO: 1 batches, avg len: 7.0


13.402337